In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 23:12:50--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Herleiden van raw.githubusercontent.com (raw.githubusercontent.com)... 100.64.1.70
Verbinding maken met raw.githubusercontent.com (raw.githubusercontent.com)|100.64.1.70|:443... verbonden.
HTTP-verzoek is verzonden; wachten op antwoord... 200 OK
Lengte: 3832 (3,7K) [text/plain]
Wordt opgeslagen als: ‘minsearch.py’

minsearch.py        100%[===================>]   3,74K  --.-KB/s    in 0s      

2024-06-27 23:12:50 (42,0 MB/s) - '‘minsearch.py’' opgeslagen [3832/3832]



In [7]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/71/7f/c626ded16e2cfdb91ab0349895ff1e4ef554aae63fd7d184ee132d269925/openai-1.35.6-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/7b/a2/10639a79341f6c019dedc95bd48a4928eed9f1d1197f4c04f546fc7ae0ff/anyio-4.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/17/ba/1b65c9cbc49e0c7cd1be086c63209e9ad883c2a409be4746c21db4263f41/pydantic-2.7.

In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [19]:
llm('write that this is a test')

' This statement serves as an initial indicator or placeholder text to signal the commencement of a system, application, or process for testing purposes. It\'s commonly used during development phases to verify if scripts are running correctly without processing any actual data or performing real operations. Here\'s how you might utilize it in code:\n\n### Example in Python:\n```python\ndef start_test():\n    print("This is a test.")\n    \nstart_test()  # Output: This is a test.\n```\n\n### Example in JavaScript (Node.js):\n```javascript\nfunction runTest() {\n    console.log("This is a test.");\n}\n\nrunTest(); // Output: This is a test.\n```\n\n### Example in Java:\n```java\npublic class TestRunner {\n    public static void main(String[] args) {\n        System.out.println("This is a test.");\n    }\n}\n// Compile with `javac TestRunner.java` and run with `./TestRunner` on UNIX systems or `java TestRunner` on Windows.\n```\n\n### Example in HTML (for web application testing):\nFor de

In [20]:
print(_)

 This statement serves as an initial indicator or placeholder text to signal the commencement of a system, application, or process for testing purposes. It's commonly used during development phases to verify if scripts are running correctly without processing any actual data or performing real operations. Here's how you might utilize it in code:

### Example in Python:
```python
def start_test():
    print("This is a test.")
    
start_test()  # Output: This is a test.
```

### Example in JavaScript (Node.js):
```javascript
function runTest() {
    console.log("This is a test.");
}

runTest(); // Output: This is a test.
```

### Example in Java:
```java
public class TestRunner {
    public static void main(String[] args) {
        System.out.println("This is a test.");
    }
}
// Compile with `javac TestRunner.java` and run with `./TestRunner` on UNIX systems or `java TestRunner` on Windows.
```

### Example in HTML (for web application testing):
For demonstration purposes, let's imagi